**Лабораторная работа №2**

**Задача:** *провести визуальный анализ данных*

Импортируем нужные нам библиотеки, скачаем данные и разделим их на тестовые и тренировочные

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier


data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', names=["Class","Alcohol","Malic acid","Ash","Alcalinity of ash","Magnesium","Total phenols","Flavanoids","Nonflavanoid phenols","Proanthocyanins","Color intensity","Hue","OD280/OD315 of diluted wines","Proline"])
data.info()

X = data[data.columns[1:]]
y = data['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)


Проверим какие признаки имееют наибольшую корреляцию с целевой переменной

In [0]:
correlations_data = data.corr()['Class'].sort_values()
print(correlations_data)

Отсечем признаки которые имеют коэффициент корреляции меньше, чем 0.6

In [0]:
data = data.drop(['Proanthocyanins', 'Alcohol', 'Magnesium', 'Ash', 'Color intensity', 'Malic acid', 'Nonflavanoid phenols', 'Alcalinity of ash'], axis='columns')

Проведем визуальный анализ наших данных

In [0]:
corr = data.corr().abs()
sns.set(rc={'figure.figsize':(12, 14)})
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [0]:
df_for_pairgrid = X_train[["Total phenols","Flavanoids","Hue","OD280/OD315 of diluted wines","Proline"]]
df_for_pairgrid['Class'] = y_train

sns.PairGrid(df_for_pairgrid, hue='Class').map(plt.scatter)

Из этих двух графиков можно сделать вывод, что теперь у нас остались признаки которые имееют определяющие значение в выборе класса.

**Задача:** *посчитать "руками" (не используя sklearn.metrics) accuracy, precision, recall и f1 для ответов, полученных на предыдущей лабе.*

Загружаем данные предыдущей лабы

In [0]:
wine = datasets.load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

knn = KNeighborsClassifier() #создание модели
knn.fit(X_train, y_train) #обучение модели
y_knn = knn.predict(X_test)
y_knn = np.around(y_knn, decimals = 0).astype(np.int64)

Считаем метрики

In [0]:
#Все относительно класса 0
TP = 0
FP = 0
FN = 0
TN = 0

for i in range(len(y_test)):
  if (y_test[i] == y_knn[i] == 0):
    TP += 1
  elif (y_test[i] != 0 and y_knn[i] == 0):
    FP += 1    
  elif (y_test[i] == 0 and y_knn[i] != 0):
    FN += 1
  else:
    TN += 1    

accuracy = (TP + TN)/(TP + TN + FP + FN)
precision = (TP)/(TP + FP)
recall = (TP)/(TP + FN)
f1 = (2 * precision * recall)/(precision + recall)

print(f"True Positive: {TP}") 
print(f"False Positive: {FP}")
print(f"False Negative: {FN}")
print(f"True Negative: {TN}\n")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")

И проверяем, что насчитал нам sklearn

In [0]:
print(classification_report(y_test, y_knn, target_names = ['class 0', 'class 1', 'class 2']))

**Задача:** *выбрать и обосновать метрику качества. Попробовать несколько методов машинного обучения из sklearn, посмотреть, какой метод лучше всего подойдет в контексте выбранной метрики (пока без подбора гиперпараметров). Оптимизировать KNN в соответствии с метрикой*

Выберем метрику precision. Нам важно, чтобы метод при внесении новых данных мог с максимальной точность определить, что за вино нам представлено. 

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score

average = 'micro'
for clf in [DecisionTreeClassifier(), SGDClassifier(), SVC(), GaussianNB(), KNeighborsClassifier()]:
    print(f"{clf.__class__.__name__}\nPrecision: {precision_score(y_test, clf.fit(X_train, y_train).predict(X_test), average=average)}\nRecall: {recall_score(y_test, clf.fit(X_train, y_train).predict(X_test), average=average)}\nF1: {f1_score(y_test, clf.fit(X_train, y_train).predict(X_test), average=average)}\n")

Из этого маленького теста видно, что нам подходит GaussianNB

Теперь оптимизируем knn в соответсвии с выбранной метрикой

In [0]:
from sklearn.model_selection import RandomizedSearchCV

params = {"n_neighbors": np.arange(1, 81),
          "weights": ["uniform", "distance"],
          "leaf_size": np.arange(1, 100)}

rsearch = RandomizedSearchCV(estimator = KNeighborsClassifier(),
                             param_distributions = params,
                             scoring = 'precision_micro',
                             n_iter = 10)
rsearch.fit(X_train, y_train)
ry_knn = rsearch.predict(X_test)
ry_knn = np.around(ry_knn, decimals = 0).astype(np.int64)

print(f"Precision: {rsearch.best_score_}")
print(f"Best params: {rsearch.best_params_}")



**Задача:** *взять любой вещественный признак (не бинарный) из своего набора данных, найти максимум значения разобранными методами оптимизации, сравнить с фактическим максимумом*

In [0]:
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution as diff

data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', names=["Class","Alcohol","Malic acid","Ash","Alcalinity of ash","Magnesium","Total phenols","Flavanoids","Nonflavanoid phenols","Proanthocyanins","Color intensity","Hue","OD280/OD315 of diluted wines","Proline"])
arr = np.array(data['Total phenols'])


def f(x, *args):
  return arr[int(x)]


x = np.arange(0, 178)
y = np.fromiter(map(f, x), dtype = float)


plt.plot(x, y)
diff(f, [(0, 178)], strategy = 'best1exp', disp = True)

In [0]:
print(f'Минимальный элемент: {arr[np.argmin(arr)]}')
print(f'Максимальный элемент: {arr[np.argmax(arr)]}')